In [12]:
# Import der benötigten Bibliotheken
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

Vorbereitung der Daten


In [13]:
# Daten-Vorbereitung
base_dir = 'C:\\Users\\Kevin Keo\\OneDrive - ZHAW\\Dokumente\\Desktop\\ZHAW WIN\\FS24\\KI-Anwendung\\Pokemon\\Mandatory Sauber\\Mandatory Sauber\\Datensätze\\pokemon'
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Behalte die Aufteilung für Training und Validation bei
)

train_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Spezifiziert, dass dies der Trainingssatz ist
)

validation_generator = datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Spezifiziert, dass dies der Validierungssatz ist
)

Found 282 images belonging to 3 classes.


Found 68 images belonging to 3 classes.


Modell Setup

In [14]:
# Modell-Setup
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Zuerst wird das Basismodell eingefroren

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout hinzugefügt, um Overfitting zu reduzieren
predictions = Dense(3, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

Training des Models

In [15]:
# Trainieren des Modells
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20,
    callbacks=[
        ModelCheckpoint('best_model.keras', save_best_only=True),
        EarlyStopping(monitor='val_loss', patience=5),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2)
    ]
)

AttributeError: module 'scipy' has no attribute 'ndimage'

Fine Tuning des Modells

In [ ]:
# Fine-Tuning des Modells
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history_fine = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 148s 12s/step - accuracy: 0.7070 - loss: 1.0460 - val_accuracy: 0.6094 - val_loss: 0.9729
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 135ms/step - accuracy: 1.0000 - loss: 0.0769 - val_accuracy: 0.5714 - val_loss: 1.0434
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 100s 10s/step - accuracy: 0.9677 - loss: 0.1108 - val_accuracy: 0.5469 - val_loss: 0.9639
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 10s 111ms/step - accuracy: 1.0000 - loss: 0.0381 - val_accuracy: 0.7143 - val_loss: 0.9019
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 96s 10s/step - accuracy: 0.9992 - loss: 0.0220 - val_accuracy: 0.2969 - val_loss: 1.1206
Epoch 6/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 164ms/step - accuracy: 1.0000 - loss: 0.0226 - val_accuracy: 0.1429 - val_loss: 1.1233
Epoch 7/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 97s 10s/step - accuracy: 1.0000 - loss: 0.0062 - val_accuracy: 0.1719 - val_loss: 1.4363
Epoch 8/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 103ms/step - accuracy: 0.9688 - loss: 0.0287 - val_accuracy: 0.1429 - val_loss

Bewertung und Ergebnisse

In [ ]:
# Ergebnisse bewerten
eval_result = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print(f'Performance vor dem Fine-Tuning: {history.history["val_accuracy"][-1]}')
print(f'Performance nach dem Fine-Tuning: {history_fine.history["val_accuracy"][-1]}')

2/2 ━━━━━━━━━━━━━━━━━━━━ 7s 3s/step - accuracy: 0.2708 - loss: 1.4508
Performance vor dem Fine-Tuning: 0.640625
Performance nach dem Fine-Tuning: 0.0


In [ ]:
model.save('mein_modell.h5') 